In [ ]:
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.6.* dask-cudf-cu12==24.6.* cuspatial-cu12==24.6.* \
    cuproj-cu12==24.6.*

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.0/478.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 22.6 MB/s eta 0

In [ ]:
pip install geopandas


In [ ]:
%%time
import cupy as cp
import cudf
import cuspatial
import json
import time
import datetime
import numpy as np
import dask_cudf
import dask.dataframe as dd
import pandas as pd
from time import strftime, localtime
import geopandas as gpd
from shapely.geometry import Polygon, Point


CPU times: user 3.24 s, sys: 365 ms, total: 3.61 s
Wall time: 5.07 s


In [ ]:
%%time
start=time.time()
columns_to_read = ['CLIMAC', 'WINDOW_START', 'POSITION.X', 'POSITION.Y']
df=dask_cudf.read_parquet('/content/drive/MyDrive/Rapids/task_1_datas/first_task.access_points_wout_tenant.parquet/27e59b19246b46d1a939fcf2a4fa72d1-0.parquet',columns=columns_to_read)
print(time.time()-start)

2.3498423099517822
CPU times: user 87.4 ms, sys: 71.5 ms, total: 159 ms
Wall time: 2.35 s


(42530002, 4)

In [ ]:
categories= ["0-15", "15-30", "30-60", "60-90", "90-120", "120+"]
special_categories= ['0-5','5-10','10-15','15-20','20+']

def get_points(my_path):
    with open(my_path) as f:
        data = json.load(f)
    coordinates = data[0]['geometry']['coordinates'][0]
    points = [Point(coord[1],coord[0]) for coord in coordinates]
    return points

def find_max_min_array(my_path):
    with open(my_path) as f:
        data = json.load(f)
    coordinates = data[0]['geometry']['coordinates'][0]
    pdf=pd.DataFrame(coordinates,columns= ['Y','X'])
    pdf= pdf[['X','Y']]
    max_x=pdf['X'].max()
    min_x=pdf['X'].min()
    max_y=pdf['Y'].max()
    min_y=pdf['Y'].min()
    return [max_x,min_x,max_y,min_y]

def first_filter(my_data_frame,my_path):
    my_array = find_max_min_array(my_path)
    condition = (my_data_frame['POSITION.X'] < my_array[0]) & \
            (my_data_frame['POSITION.X'] > my_array[1]) & \
            (my_data_frame['POSITION.Y'] < my_array[2]) & \
            (my_data_frame['POSITION.Y'] > my_array[3])
    return my_data_frame.loc[condition]

def filter_within_polygon_partition(df, polygon_points):
    # Convert the points DataFrame to numpy arrays

    points_xy = cp.array(df[['POSITION.X', 'POSITION.Y']].to_cupy().ravel())

    # Create GeoSeries for points directly from CuPy arrays
    points = cuspatial.GeoSeries.from_points_xy(points_xy)

    # Create a GeoSeries for the polygon with swapped coordinates

    polygon = cuspatial.GeoSeries([Polygon(polygon_points)])

    # Apply the cuspatial point_in_polygon method
    df = df.assign(within_polygon=cuspatial.point_in_polygon(points, polygon))

    # Filter the DataFrame
    filtered_df = df[df['within_polygon']].drop(columns='within_polygon')

    return filtered_df

def filter_within_polygon(my_data_frame, my_path):
    polygon_points = get_points(my_path)  # Assuming this returns the polygon points

    # Apply the filtering function to each partition
    my_filtered_data_frame = my_data_frame.map_partitions(filter_within_polygon_partition, polygon_points)

    return my_filtered_data_frame


def convert_date_time(df):

    df['WINDOW_START'] = df['WINDOW_START'] + 10800
    df['WINDOW_START_DATETIME'] = df['WINDOW_START'].astype('datetime64[s]')

    # Extract date and time components
    df['WINDOW_START_DATE'] = df['WINDOW_START_DATETIME'].dt.strftime('%Y-%m-%d')
    df['WINDOW_START_TIME'] = df['WINDOW_START_DATETIME'].dt.strftime('%H:%M:%S')
    return df

def group_df(my_filtered_df, my_column_name):
    original_dtypes = {'CLIMAC': ('string'), 'WINDOW_START_DATE': ('object'), my_column_name: ('float64')}
    grouped_df = my_filtered_df.groupby(['CLIMAC','WINDOW_START_DATE']).size().reset_index()
    grouped_df.columns = ['CLIMAC', 'WINDOW_START_DATE','MEMBER_COUNT']
    grouped_df[my_column_name] = grouped_df['MEMBER_COUNT'] / 2
    grouped_df=grouped_df.drop(columns='MEMBER_COUNT')
    grouped_df = grouped_df.astype(original_dtypes)
    return grouped_df

def selection_cudf(s):
    out = cudf.Series([""] * len(s), dtype='object')
    out[s <= 15] = categories[0]
    out[(s > 15) & (s <= 30)] = categories[1]
    out[(s > 30) & (s <= 60)] = categories[2]
    out[(s > 60) & (s <= 90)] = categories[3]
    out[(s > 90) & (s <= 120)] = categories[4]
    out[s > 120] = categories[5]
    return out

def special_selection_cudf(s):
    out = cudf.Series([""] * len(s), dtype='object')
    out[s <= 5] = special_categories[0]
    out[(s > 5) & (s <= 10)] = special_categories[1]
    out[(s > 10) & (s <= 15)] = special_categories[2]
    out[(s > 15) & (s <= 20)] = special_categories[3]
    out[s > 20] = special_categories[4]
    return out

def categorization(my_df, my_column_name, my_selection_column_name):
    my_df[my_column_name] = my_df[my_selection_column_name].map_partitions(selection_cudf, meta=('x', 'str'))
    return my_df

def special_categorization(my_df):
    my_df['FIDS_SPECIAL_CATEGORY'] = my_df['FIDS_TOTAL_TIME_IN_MINUTE'].map_partitions(special_selection_cudf, meta=('x', 'str'))
    return my_df

In [ ]:
%%time
filtered_fids_df = filter_within_polygon(df, '/content/drive/MyDrive/Rapids/task_1_datas/wifi.fids_coordinates.json')
filtered_shop_df = filter_within_polygon(df, '/content/drive/MyDrive/Rapids/task_1_datas/wifi.shop_area_coordinates.json')
filtered_east_df = filter_within_polygon(df, '/content/drive/MyDrive/Rapids/task_1_datas/wifi.shop_area_east_coordinates.json')
filtered_west_df = filter_within_polygon(df, '/content/drive/MyDrive/Rapids/task_1_datas/wifi.shop_area_west_coordinates.json')

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


CPU times: user 823 ms, sys: 55 ms, total: 878 ms
Wall time: 3.81 s


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [ ]:
filtered_east_df.compute().shape

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


(3386675, 7)

In [ ]:
%%time
filtered_fids_df = convert_date_time(filtered_fids_df)
filtered_shop_df = convert_date_time(filtered_shop_df)
filtered_east_df = convert_date_time(filtered_east_df)
filtered_west_df = convert_date_time(filtered_west_df)

CPU times: user 31.4 ms, sys: 942 µs, total: 32.3 ms
Wall time: 31.7 ms


In [ ]:
%%time
grouped_fids_df = group_df(filtered_fids_df,'FIDS_TOTAL_TIME_IN_MINUTE')
grouped_shop_df = group_df(filtered_shop_df,'SHA_TOTAL_TIME_IN_MINUTE')
grouped_east_df = group_df(filtered_east_df,'SHA_EAST_TOTAL_TIME_IN_MINUTE')
grouped_west_df = group_df(filtered_west_df,'SHA_WEST_TOTAL_TIME_IN_MINUTE')



CPU times: user 74.3 ms, sys: 8.63 ms, total: 82.9 ms
Wall time: 87.3 ms


In [ ]:
%%time
grouped_fids_df=categorization(grouped_fids_df,'FIDS_CATEGORY','FIDS_TOTAL_TIME_IN_MINUTE')
grouped_fids_df=special_categorization(grouped_fids_df)

grouped_shop_df=categorization(grouped_shop_df,'SHA_CATEGORY','SHA_TOTAL_TIME_IN_MINUTE')
grouped_east_df=categorization(grouped_east_df,'SHA_EAST_CATEGORY','SHA_EAST_TOTAL_TIME_IN_MINUTE')
grouped_west_df=categorization(grouped_west_df,'SHA_WEST_CATEGORY','SHA_WEST_TOTAL_TIME_IN_MINUTE')

CPU times: user 17 ms, sys: 0 ns, total: 17 ms
Wall time: 16.6 ms


In [ ]:
%%time
two_merged_df= grouped_fids_df.merge(grouped_shop_df,how='inner', on=['CLIMAC','WINDOW_START_DATE'])

three_merged_df= two_merged_df.merge(grouped_east_df, on=['CLIMAC','WINDOW_START_DATE'],how='left')

four_merged_df= three_merged_df.merge(grouped_west_df, on=['CLIMAC','WINDOW_START_DATE'],how='left')

CPU times: user 26.9 ms, sys: 4.86 ms, total: 31.8 ms
Wall time: 31.5 ms


In [ ]:
four_merged_df.compute()

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


,CLIMAC,WINDOW_START_DATE,FIDS_TOTAL_TIME_IN_MINUTE,FIDS_CATEGORY,FIDS_SPECIAL_CATEGORY,SHA_TOTAL_TIME_IN_MINUTE,SHA_CATEGORY,SHA_EAST_TOTAL_TIME_IN_MINUTE,SHA_EAST_CATEGORY,SHA_WEST_TOTAL_TIME_IN_MINUTE,SHA_WEST_CATEGORY
0,ea:f5:ea:fd:b0:d7,2024-05-24,1.5,0-15,0-5,9.0,0-15,3.5,0-15,1.0,0-15
1,6e:90:9e:47:34:07,2024-05-23,1.0,0-15,0-5,176.5,120+,140.0,120+,32.5,30-60
2,3a:30:df:ef:98:92,2024-05-25,1.5,0-15,0-5,14.5,0-15,6.0,0-15,7.0,0-15
3,26:71:81:67:33:b4,2024-05-25,28.0,15-30,20+,37.0,30-60,3.5,0-15,16.5,15-30
4,50:8e:49:d5:aa:2b,2024-05-23,6.0,0-15,5-10,13.5,0-15,4.5,0-15,2.5,0-15
...,...,...,...,...,...,...,...,...,...,...,...
34605,02:a9:75:b0:68:28,2024-05-23,0.5,0-15,0-5,4.5,0-15,<NA>,<NA>,4.5,0-15
34606,3e:6b:23:da:8f:3b,2024-05-24,1.0,0-15,0-5,8.0,0-15,5.5,0-15,2.5,0-15
34607,c6:2e:14:e6:fe:07,2024-05-24,6.0,0-15,5-10,6.0,0-15,<NA>,<NA>,<NA>,<NA>
34608,22:d8:8d:3e:00:94,2024-05-24,1.0,0-15,0-5,6.0,0-15,<NA>,<NA>,6.0,0-15
